In [183]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import models
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping
import ast
import pickle
import optuna

Most of this code comes from Googles Training course on text classification:  https://developers.google.com/machine-learning/guides/text-classification.  This has been adapted and improved to include k-folds cross validation to produce a more consistent result in training.  Given the volume of training data is low.

### Step 1 - Import Train and Test Data and convert to required data type

In [138]:
# Load training set from CSV, convert CSV text to list and join into a single string
train_data = pd.read_csv("train_data.csv")
train_data['preprocessed_text'] = train_data['preprocessed_text'].apply(ast.literal_eval)
train_data['preprocessed_text'] = train_data['preprocessed_text'].apply(' '.join)
train_texts = train_data['preprocessed_text'].to_numpy()

# Load test set from CSV, convert CSV text to list and join into a single string
test_data = pd.read_csv("test_data.csv")
test_data['preprocessed_text'] = test_data['preprocessed_text'].apply(ast.literal_eval)
test_data['preprocessed_text'] = test_data['preprocessed_text'].apply(' '.join)
test_texts = test_data['preprocessed_text'].to_numpy()


In [166]:
# Create dictionary of integer values for the labels so we can convert back at a later dat
def create_label_dict(df, col_name):
    unique_values = df[col_name].unique()
    value_dict = {value: i for i, value in enumerate(unique_values)}
    return value_dict

label_dict = create_label_dict(train_data, 'clause_type')

# convert existing text labels to integers 1-10 using dictionary
def transform_column(df, col_name, mapping_dict):
    return df[col_name].map(mapping_dict).values.astype(np.float32)

train_labels = transform_column(train_data, 'clause_type', label_dict)
test_labels = transform_column(test_data, 'clause_type', label_dict)


### Step 2 - Functions for Vectorisation

In [175]:
# Vectorization parameters
# Range (inclusive) of n-gram sizes for tokenizing text.
NGRAM_RANGE = (1, 2)
# Limit on the number of features. We use the top 20K features.
TOP_K = 20000
# Whether text should be split into word or character n-grams. One of 'word', 'char'.
TOKEN_MODE = 'word'
# Minimum document/corpus frequency below which a token will be discarded.
MIN_DOCUMENT_FREQUENCY = 2

def ngram_vectorize(x_train_fold, y_train_fold, x_val_fold):
    """Vectorizes texts as n-gram vectors.

    1 text = 1 tf-idf vector the length of vocabulary of unigrams + bigrams.

    # Arguments
        x_train_fold: list, training text strings.
        y_train_fold: np.ndarray, training labels (integers).
        x_val_fold: list, validation text strings.

    # Returns
        x_train: vectorized training texts
        x_val: vectorized validation texts
    """
    # Create keyword arguments to pass to the 'tf-idf' vectorizer.
    kwargs = {
        'ngram_range': NGRAM_RANGE,  # Use 1-grams + 2-grams.
        'dtype': 'int32',
        'strip_accents': 'unicode',
        'decode_error': 'replace',
        'analyzer': TOKEN_MODE,  # Split text into word tokens.
        'min_df': MIN_DOCUMENT_FREQUENCY,
    }
    vectorizer = TfidfVectorizer(**kwargs)

    # Learn vocabulary from training texts and vectorize training texts.
    x_train_fold = vectorizer.fit_transform(x_train_fold).astype(np.float32)  # Fixed variable name

    # Vectorize validation texts.
    x_val_fold = vectorizer.transform(x_val_fold).astype(np.float32)

    # Select top 'k' of the vectorized features.
    selector = SelectKBest(f_classif, k=min(TOP_K, x_train_fold.shape[1]))
    selector.fit(x_train_fold, y_train_fold)
    x_train = selector.transform(x_train_fold).astype('float32')
    x_val = selector.transform(x_val_fold).astype('float32')
    
    # Save vectorizer and selector to disk for future use
    with open('vectorizer.pkl', 'wb') as f:
        pickle.dump(vectorizer, f)

    with open('selector.pkl', 'wb') as f:
        pickle.dump(selector, f)
    
    # Return both the vectorized texts
    return x_train, x_val

In [176]:
def vectorize_production_data(prod_texts):
    """Vectorizes new production texts using the pre-trained vectorizer and selector.

    # Arguments
        prod_texts: list, new production text strings.

    # Returns
        x_prod: vectorized production texts.
    """

    # Load the saved vectorizer and feature selector
    with open('vectorizer.pkl', 'rb') as f:
        vectorizer = pickle.load(f)

    with open('selector.pkl', 'rb') as f:
        selector = pickle.load(f)

    # Vectorize production texts using the trained vectorizer
    x_prod = vectorizer.transform(prod_texts).astype(np.float32)

    # Apply feature selection using the trained selector
    x_prod = selector.transform(x_prod).astype(np.float32)

    return x_prod

### Step 3 - Define Function to create last layer according to data parameters

In [54]:
def _get_last_layer_units_and_activation(num_classes):
    """Gets the # units and activation function for the last network layer.
    # Arguments
        num_classes: int, number of classes.
    # Returns
        units, activation values.
    """
    if num_classes == 2:
        activation = 'sigmoid'
        units = 1
    else:
        activation = 'softmax'
        units = num_classes
    return units, activation

### Step 4 - Define function to build simple multi-layer perceptron  

In [169]:
def mlp_model(layers, units, dropout_rate, input_shape, num_classes):
    """Creates an instance of a multi-layer perceptron model.

    # Arguments
        layers: int, number of `Dense` layers in the model.
        units: int, output dimension of the layers.
        dropout_rate: float, percentage of input to drop at Dropout layers.
        input_shape: tuple, shape of input to the model.
        num_classes: int, number of output classes.

    # Returns
        An MLP model instance.
    """
    op_units, op_activation = _get_last_layer_units_and_activation(num_classes)
    model = models.Sequential()

    # Define the input shape using Input layer
    model.add(Input(shape=input_shape))

    # Add the first Dense layer (input layer) without Dropout yet
    model.add(Dense(units=units, activation='relu'))

    # Add Dropout and subsequent layers
    for _ in range(layers - 1):
        model.add(Dropout(rate=dropout_rate))  # Add Dropout after Dense layers
        model.add(Dense(units=units, activation='relu'))

    # Output layer
    model.add(Dense(units=op_units, activation=op_activation))

    return model

### Step 5 - Create a function to train the n_gram model

In [163]:
# train model with kfolds validation
def train_ngram_model_kfolds(data,k=5,
                      learning_rate=1e-3,
                      epochs=1000,
                      batch_size=128,
                      layers=2,
                      units=64,
                      dropout_rate=0.2,
                      num_classes=10):

    """Trains n-gram model on the given dataset.

    # Arguments
        k: no of folds 
        data: tuples of training and test texts and labels.
        learning_rate: float, learning rate for training model.
        epochs: int, number of epochs.
        batch_size: int, number of samples per batch.
        layers: int, number of `Dense` layers in the model.
        units: int, output dimension of Dense layers in the model.
        dropout_rate: float: percentage of input to drop at Dropout layers.
        num_classes: int, number of output classes.
    """
    # Get the data.
    train_texts, train_labels = data
    
    # Prepare k-fold cross-validation
    kfold = KFold(n_splits=k, shuffle=True, random_state=42)
    
    # Store validation results for each fold
    val_accuracies = []
    val_losses = []

    # Perform K-Fold Cross-Validation
    for train_idx, val_idx in kfold.split(train_texts):
        # Split data into training and validation for this fold
        # K-fold cross-validation
        x_train_fold = train_texts[train_idx]
        y_train_fold = train_labels[train_idx]
        x_val_fold = train_texts[val_idx]
        y_val_fold = train_labels[val_idx]
        
        # Vectorize texts.
        x_train, x_val = ngram_vectorize(x_train_fold, y_train_fold, x_val_fold) 
        # Create model instance.
        model = mlp_model(layers=layers,
                          units=units,
                          dropout_rate=dropout_rate,
                          input_shape=x_train.shape[1:],
                          num_classes=num_classes)
        
        loss = 'sparse_categorical_crossentropy'
        optimizer = Adam(learning_rate=learning_rate)
        model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    
        # Create callback for early stopping on validation loss. If the loss does not decrease in two consecutive tries, stop training.
        callbacks = [EarlyStopping(monitor='val_loss', patience=2)]
        
        # Train and validate the model on this fold
        history = model.fit(
            x_train,
            y_train_fold,
            epochs=epochs,
            callbacks=callbacks,
            validation_data=(x_val, y_val_fold),
            verbose=2,
            batch_size=batch_size
        )
        # Collect the last validation accuracy and loss
        history = history.history
        val_accuracies.append(history['val_accuracy'][-1])
        val_losses.append(history['val_loss'][-1])
    
    # Compute average validation accuracy and loss
    avg_val_accuracy = np.mean(val_accuracies)
    avg_val_loss = np.mean(val_losses)
    
    # Print results.
    print('Average Validation Accuracy: {:.4f}, Average Loss: {:.4f}'.format(avg_val_accuracy, avg_val_loss))

    # Save model.
    model.save('Clause_class_model.keras')
    
    return avg_val_accuracy, avg_val_loss

### Step 6 - Build Model and train model

In [178]:
data = (train_texts, train_labels)
train_ngram_model_kfolds(data)

/Users/iangosling/PycharmProjects/Imp ML Course/.venv/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:2030: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  warnings.warn(


Epoch 1/1000
11/11 - 1s - 49ms/step - accuracy: 0.4143 - loss: 2.2660 - val_accuracy: 0.7183 - val_loss: 2.1738
Epoch 2/1000
11/11 - 0s - 9ms/step - accuracy: 0.7347 - loss: 2.0875 - val_accuracy: 0.7461 - val_loss: 1.9355
Epoch 3/1000
11/11 - 0s - 9ms/step - accuracy: 0.7921 - loss: 1.8088 - val_accuracy: 0.7771 - val_loss: 1.6120
Epoch 4/1000
11/11 - 0s - 9ms/step - accuracy: 0.8425 - loss: 1.4546 - val_accuracy: 0.7740 - val_loss: 1.2606
Epoch 5/1000
11/11 - 0s - 9ms/step - accuracy: 0.8860 - loss: 1.0837 - val_accuracy: 0.8297 - val_loss: 0.9403
Epoch 6/1000
11/11 - 0s - 9ms/step - accuracy: 0.9325 - loss: 0.7584 - val_accuracy: 0.8638 - val_loss: 0.7018
Epoch 7/1000
11/11 - 0s - 9ms/step - accuracy: 0.9496 - loss: 0.5192 - val_accuracy: 0.8576 - val_loss: 0.5578
Epoch 8/1000
11/11 - 0s - 8ms/step - accuracy: 0.9503 - loss: 0.3711 - val_accuracy: 0.8638 - val_loss: 0.4803
Epoch 9/1000
11/11 - 0s - 9ms/step - accuracy: 0.9542 - loss: 0.2774 - val_accuracy: 0.8607 - val_loss: 0.4409


/Users/iangosling/PycharmProjects/Imp ML Course/.venv/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:2030: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  warnings.warn(


Epoch 1/1000
11/11 - 0s - 44ms/step - accuracy: 0.6121 - loss: 2.2587 - val_accuracy: 0.7678 - val_loss: 2.1575
Epoch 2/1000
11/11 - 0s - 9ms/step - accuracy: 0.8332 - loss: 2.0549 - val_accuracy: 0.7678 - val_loss: 1.8904
Epoch 3/1000
11/11 - 0s - 9ms/step - accuracy: 0.8425 - loss: 1.7321 - val_accuracy: 0.7740 - val_loss: 1.5306
Epoch 4/1000
11/11 - 0s - 9ms/step - accuracy: 0.8534 - loss: 1.3288 - val_accuracy: 0.7864 - val_loss: 1.1451
Epoch 5/1000
11/11 - 0s - 9ms/step - accuracy: 0.9046 - loss: 0.9281 - val_accuracy: 0.8452 - val_loss: 0.8303
Epoch 6/1000
11/11 - 0s - 9ms/step - accuracy: 0.9255 - loss: 0.6375 - val_accuracy: 0.8700 - val_loss: 0.6336
Epoch 7/1000
11/11 - 0s - 9ms/step - accuracy: 0.9418 - loss: 0.4353 - val_accuracy: 0.8916 - val_loss: 0.5163
Epoch 8/1000
11/11 - 0s - 10ms/step - accuracy: 0.9558 - loss: 0.3292 - val_accuracy: 0.8947 - val_loss: 0.4546
Epoch 9/1000
11/11 - 0s - 9ms/step - accuracy: 0.9635 - loss: 0.2519 - val_accuracy: 0.8854 - val_loss: 0.4250

/Users/iangosling/PycharmProjects/Imp ML Course/.venv/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:2030: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  warnings.warn(


...

/Users/iangosling/PycharmProjects/Imp ML Course/.venv/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:2030: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  warnings.warn(


Epoch 1/1000
11/11 - 0s - 42ms/step - accuracy: 0.4783 - loss: 2.2577 - val_accuracy: 0.6801 - val_loss: 2.1541
Epoch 2/1000
11/11 - 0s - 9ms/step - accuracy: 0.7938 - loss: 2.0560 - val_accuracy: 0.7702 - val_loss: 1.9055
Epoch 3/1000
11/11 - 0s - 9ms/step - accuracy: 0.8450 - loss: 1.7555 - val_accuracy: 0.7950 - val_loss: 1.5727
Epoch 4/1000
11/11 - 0s - 9ms/step - accuracy: 0.8705 - loss: 1.3715 - val_accuracy: 0.8043 - val_loss: 1.2069
Epoch 5/1000
11/11 - 0s - 9ms/step - accuracy: 0.9062 - loss: 0.9659 - val_accuracy: 0.8168 - val_loss: 0.8829
Epoch 6/1000
11/11 - 0s - 9ms/step - accuracy: 0.9426 - loss: 0.6438 - val_accuracy: 0.8354 - val_loss: 0.6689
Epoch 7/1000
11/11 - 0s - 9ms/step - accuracy: 0.9488 - loss: 0.4520 - val_accuracy: 0.8385 - val_loss: 0.5579
Epoch 8/1000
11/11 - 0s - 10ms/step - accuracy: 0.9566 - loss: 0.3360 - val_accuracy: 0.8540 - val_loss: 0.4954
Epoch 9/1000
11/11 - 0s - 10ms/step - accuracy: 0.9667 - loss: 0.2538 - val_accuracy: 0.8478 - val_loss: 0.470

/Users/iangosling/PycharmProjects/Imp ML Course/.venv/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:2030: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  warnings.warn(


Epoch 1/1000
11/11 - 1s - 68ms/step - accuracy: 0.5209 - loss: 2.2587 - val_accuracy: 0.7919 - val_loss: 2.1523
Epoch 2/1000
11/11 - 0s - 9ms/step - accuracy: 0.8326 - loss: 2.0585 - val_accuracy: 0.8261 - val_loss: 1.8853
Epoch 3/1000
11/11 - 0s - 9ms/step - accuracy: 0.8775 - loss: 1.7474 - val_accuracy: 0.8385 - val_loss: 1.5231
Epoch 4/1000
11/11 - 0s - 9ms/step - accuracy: 0.9132 - loss: 1.3409 - val_accuracy: 0.8540 - val_loss: 1.1184
Epoch 5/1000
11/11 - 0s - 9ms/step - accuracy: 0.9256 - loss: 0.9412 - val_accuracy: 0.8602 - val_loss: 0.7915
Epoch 6/1000
11/11 - 0s - 9ms/step - accuracy: 0.9349 - loss: 0.6328 - val_accuracy: 0.8696 - val_loss: 0.5848
Epoch 7/1000
11/11 - 0s - 11ms/step - accuracy: 0.9442 - loss: 0.4435 - val_accuracy: 0.8820 - val_loss: 0.4733
Epoch 8/1000
11/11 - 0s - 9ms/step - accuracy: 0.9519 - loss: 0.3293 - val_accuracy: 0.8820 - val_loss: 0.4108
Epoch 9/1000
11/11 - 0s - 9ms/step - accuracy: 0.9589 - loss: 0.2493 - val_accuracy: 0.8696 - val_loss: 0.3904

(0.8684825778007508, 0.39200614094734193)

### Step 7 - Make predictions on test data prior to Hyperparameter Optimisation

In [185]:
# Vectorize the production data using the previously defined function
x_prod = vectorize_production_data(test_texts)

# Make predictions on the production data using the trained model
model = load_model('Clause_class_model.keras')
predictions = model.predict(x_prod)

# Step 3: Convert predicted probabilities to class labels. N.B. Its argmax because you get a softmax output so you want the most probab;y class
predicted_classes = np.argmax(predictions, axis=1)

# Step 4: Calculate accuracy by comparing predicted classes with true labels
accuracy = accuracy_score(test_labels, predicted_classes)

# Step 5: Output the accuracy
print(f"Accuracy on test data: {accuracy:.4f}")

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Accuracy on test data: 0.8885


### Step 8 - Tune Hyperparameters

In [188]:
def objective(trial):
    data = (train_texts, train_labels)
    # Define the hyperparameters to optimize
    learning_rate = trial.suggest_float("learning_rate", 1e-4,1e-1)
    layers = trial.suggest_int("layers", 2, 6)
    units = trial.suggest_categorical("units", [32,64,128])
    dropout_rate = trial.suggest_float("dropout_rate", 0.2, 0.5)
    
    # Call the train_ngram_model_kfolds function with the sampled hyperparameters
    avg_val_accuracy, avg_val_loss = train_ngram_model_kfolds(data, learning_rate=learning_rate,layers=layers,units=units,dropout_rate=dropout_rate,num_classes=10)

    # Since Optuna minimizes the objective, we may want to minimize -accuracy if maximizing accuracy
    return avg_val_accuracy  # Maximize accuracy by minimizing negative accuracy

# Create an Optuna study to optimize the objective function
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

# Print the best result
print("Best value:", study.best_value)
print("Best parameters:", study.best_params)

[I 2024-09-20 18:26:01,102] A new study created in memory with name: no-name-1a27ab65-c32a-4aa1-8ba4-0a3710f4acc5
/Users/iangosling/PycharmProjects/Imp ML Course/.venv/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:2030: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  warnings.warn(


Epoch 1/1000
11/11 - 1s - 74ms/step - accuracy: 0.2715 - loss: 1.9256 - val_accuracy: 0.5263 - val_loss: 1.1657
Epoch 2/1000
11/11 - 0s - 7ms/step - accuracy: 0.4577 - loss: 1.4121 - val_accuracy: 0.6099 - val_loss: 0.9502
Epoch 3/1000
11/11 - 0s - 8ms/step - accuracy: 0.5120 - loss: 1.2825 - val_accuracy: 0.6409 - val_loss: 0.9162
Epoch 4/1000
11/11 - 0s - 7ms/step - accuracy: 0.5369 - loss: 1.0726 - val_accuracy: 0.6594 - val_loss: 0.8944
Epoch 5/1000
11/11 - 0s - 8ms/step - accuracy: 0.5943 - loss: 0.9705 - val_accuracy: 0.6811 - val_loss: 0.7414
Epoch 6/1000
11/11 - 0s - 7ms/step - accuracy: 0.6268 - loss: 0.9192 - val_accuracy: 0.7678 - val_loss: 0.5959
Epoch 7/1000
11/11 - 0s - 7ms/step - accuracy: 0.6129 - loss: 1.0034 - val_accuracy: 0.7337 - val_loss: 0.6476
Epoch 8/1000
11/11 - 0s - 7ms/step - accuracy: 0.6517 - loss: 0.8244 - val_accuracy: 0.7802 - val_loss: 0.5636
Epoch 9/1000
11/11 - 0s - 7ms/step - accuracy: 0.6517 - loss: 0.8042 - val_accuracy: 0.7833 - val_loss: 0.5375


/Users/iangosling/PycharmProjects/Imp ML Course/.venv/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:2030: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  warnings.warn(


Epoch 1/1000
11/11 - 1s - 65ms/step - accuracy: 0.2420 - loss: 2.0088 - val_accuracy: 0.4644 - val_loss: 1.4298
Epoch 2/1000
11/11 - 0s - 7ms/step - accuracy: 0.3887 - loss: 1.5946 - val_accuracy: 0.6533 - val_loss: 1.0578
Epoch 3/1000
11/11 - 0s - 7ms/step - accuracy: 0.4965 - loss: 1.2677 - val_accuracy: 0.5666 - val_loss: 0.8310
Epoch 4/1000
11/11 - 0s - 7ms/step - accuracy: 0.5423 - loss: 1.1168 - val_accuracy: 0.5759 - val_loss: 0.8341
Epoch 5/1000
11/11 - 0s - 8ms/step - accuracy: 0.5454 - loss: 1.0765 - val_accuracy: 0.6904 - val_loss: 0.7751
Epoch 6/1000
11/11 - 0s - 7ms/step - accuracy: 0.5702 - loss: 1.0200 - val_accuracy: 0.6718 - val_loss: 0.7837
Epoch 7/1000
11/11 - 0s - 7ms/step - accuracy: 0.6284 - loss: 0.9261 - val_accuracy: 0.7461 - val_loss: 0.6544
Epoch 8/1000
11/11 - 0s - 8ms/step - accuracy: 0.6493 - loss: 0.9480 - val_accuracy: 0.6780 - val_loss: 0.6234
Epoch 9/1000
11/11 - 0s - 7ms/step - accuracy: 0.6726 - loss: 0.7890 - val_accuracy: 0.7554 - val_loss: 0.5390


/Users/iangosling/PycharmProjects/Imp ML Course/.venv/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:2030: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  warnings.warn(


...

Epoch 1/1000
11/11 - 1s - 61ms/step - accuracy: 0.1543 - loss: 2.2100 - val_accuracy: 0.2919 - val_loss: 1.8277
Epoch 2/1000
11/11 - 0s - 8ms/step - accuracy: 0.3116 - loss: 1.8033 - val_accuracy: 0.6025 - val_loss: 1.3476
Epoch 3/1000
11/11 - 0s - 7ms/step - accuracy: 0.4721 - loss: 1.3963 - val_accuracy: 0.7143 - val_loss: 0.8495
Epoch 4/1000
11/11 - 0s - 7ms/step - accuracy: 0.5729 - loss: 1.1355 - val_accuracy: 0.6770 - val_loss: 0.7199
Epoch 5/1000
11/11 - 0s - 7ms/step - accuracy: 0.6147 - loss: 0.9961 - val_accuracy: 0.6988 - val_loss: 0.6556
Epoch 6/1000
11/11 - 0s - 7ms/step - accuracy: 0.6395 - loss: 0.9067 - val_accuracy: 0.6832 - val_loss: 0.6000
Epoch 7/1000
11/11 - 0s - 7ms/step - accuracy: 0.6574 - loss: 0.8210 - val_accuracy: 0.7609 - val_loss: 0.5700
Epoch 8/1000
11/11 - 0s - 7ms/step - accuracy: 0.6481 - loss: 0.8767 - val_accuracy: 0.7733 - val_loss: 0.5925
Epoch 9/1000
11/11 - 0s - 7ms/step - accuracy: 0.6729 - loss: 0.8121 - val_accuracy: 0.7857 - val_loss: 0.5414


/Users/iangosling/PycharmProjects/Imp ML Course/.venv/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:2030: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  warnings.warn(


Epoch 1/1000
11/11 - 7s - 592ms/step - accuracy: 0.1411 - loss: 2.2063 - val_accuracy: 0.6025 - val_loss: 1.7441
Epoch 2/1000
11/11 - 0s - 10ms/step - accuracy: 0.3419 - loss: 1.7852 - val_accuracy: 0.4876 - val_loss: 1.3286
Epoch 3/1000
11/11 - 0s - 8ms/step - accuracy: 0.4349 - loss: 1.4882 - val_accuracy: 0.5807 - val_loss: 1.0574
Epoch 4/1000
11/11 - 0s - 7ms/step - accuracy: 0.4744 - loss: 1.2855 - val_accuracy: 0.6894 - val_loss: 0.8784
Epoch 5/1000
11/11 - 0s - 8ms/step - accuracy: 0.5473 - loss: 1.1634 - val_accuracy: 0.7298 - val_loss: 0.7579
Epoch 6/1000
11/11 - 0s - 8ms/step - accuracy: 0.5729 - loss: 1.0330 - val_accuracy: 0.7081 - val_loss: 0.6847
Epoch 7/1000
11/11 - 0s - 8ms/step - accuracy: 0.6000 - loss: 0.9979 - val_accuracy: 0.7764 - val_loss: 0.6253
Epoch 8/1000
11/11 - 0s - 10ms/step - accuracy: 0.6186 - loss: 0.8693 - val_accuracy: 0.7298 - val_loss: 0.5918
Epoch 9/1000
11/11 - 0s - 9ms/step - accuracy: 0.6380 - loss: 0.8170 - val_accuracy: 0.7609 - val_loss: 0.54

[I 2024-09-20 18:44:24,235] Trial 99 finished with value: 0.7444512724876404 and parameters: {'learning_rate': 0.010310748119187137, 'layers': 4, 'units': 32, 'dropout_rate': 0.4778413486170904}. Best is trial 71 with value: 0.8765475153923035.


Best value: 0.8765475153923035
Best parameters: {'learning_rate': 0.0022575909698850163, 'layers': 3, 'units': 64, 'dropout_rate': 0.4899016759235707}


### Step 10 - Test Model
